In [114]:
import pymongo
import plotly.express as px
import pandas as pd
from owlready2 import *

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["MMM_Task"]
meta_col = mydb["MMM_Task.meta"]
time_col = mydb["MMM_G26_T"]

In [6]:
def get_trajectory(ids, start_time, end_time):
    result = time_col.aggregate([
        {
            "$match": {"timestamp": {"$lte": end_time, "$gte": start_time}}
        },
        {
            "$project" : {
                "timestamp": 1, 
                "individuals": {
                    "$filter": {
                        "input": "$individuals",
                        "as": "individual",
                        "cond": {"$in": ["$$individual.id", ids]}
                    }
                }
            }
        },
        {
            "$unwind": "$individuals"
        },
        {
            "$project" : {"id": "$individuals.id", "timestamp": 1, "loc": "$individuals.loc"}
        },
        {
            "$group": 
            {
                "_id": "$id",
                "locations": {"$push": {"x": "$loc.x", "y": "$loc.y", "z": "$loc.z", "time": "$timestamp"}}
            }
        },
            {
            "$project": {"_id": 0, "id": "$_id", "timestamps": 1, "locations": 
                         {
                             "$map": {
                                 "input": "$locations",
                                 "as": "location",
                                 "in": ["$$location.x", "$$location.y","$$location.z", "$$location.time"]
                             }
                         }
                        }
        },
        {
            "$lookup":
            {
                "from": "MMM_Task.meta",
                "let": {"id": "$id"},
                "pipeline": 
                [
                    {"$unwind": "$individuals"},
                    {"$project": 
                     {
                         "_id": "$individuals.id",
                         "className": "$individuals.class",
                     }
                    },
                    {"$match":{"$expr": {"$eq": ["$_id", "$$id"]}}},
                ],
                "as": "className"
            }
        },
        {
            "$project": {"id": 1, "locations": 1, "className": 
                         {
                             "$first": "$className"
                         }
                        }
        },
        {
            "$project": {"id": 1, "locations": 1, "className": "$className.className"
                        }
        }
    ])
    return result

In [115]:
# Works with a List of Object IDs and plots the trajectory of each object in a plot in it's own color
def plot_trajectories(object_ids, start_time=0, end_time=sys.maxsize):
    pf = pd.DataFrame(columns=['X', 'Y', 'Z', 'Time', 'ID'])
    
    data = list(get_trajectory(object_ids, start_time, end_time))
    
    for individual in data:
        individual_id = individual["id"]
        individual_class = individual["className"]
        trajectory = individual["locations"]
        df =  pd.DataFrame(trajectory, columns=['X', 'Y', 'Z', 'Time'])
        df['ID'] = f"{individual_class}: {individual_id}"
        pf = pd.concat([df, pf])
    
      
    fig = px.line_3d(pf, x='X', y='Y', z='Z', color='ID', hover_name="Time", width=1000, height=600)
    # fig = px.scatter_3d(df, x='X', y='Y', z='Z',color='Time')
    fig.show()

In [116]:
onto_path.append("./SL_Files/")
onto = get_ontology("file:///Users/timonstahlbock/Desktop/VR-LL-G2/sem_log_scripts/SL_Files/Tasks/MMM_Task/MMM_G26_T_ED.owl")
onto.load()

get_ontology("http://knowrob.org/kb/ameva_log.owl#")

In [106]:
touching_situations = []
for event in onto.individuals():
    
    # print(event.get_properties())
    # print(*i.get_inverse_properties())

    if ((event.is_a[0]).name == "TouchingSituation"):
        event_data = {}
        event_data["start_time"] = float((event.startTime[0]).name.replace("timepoint_", ""))
        event_data["end_time"] = float((event.endTime[0]).name.replace("timepoint_", ""))
        event_data["objects"] =  [obj.replace("http://knowrob.org/kb/ameva_log.owl#", "") for obj in list(event.inContact)]
        touching_situations.append(event_data)
        
print(touching_situations)

[{'start_time': 0.476154, 'end_time': 0.907067, 'objects': ['AuP-SAqPu02Vrgu6df6Txg', '9jrJzTZvSkqUjwIvPEavYA']}, {'start_time': 0.476154, 'end_time': 1.017507, 'objects': ['BENamAV8rkibLIBc8asHhQ', '9jrJzTZvSkqUjwIvPEavYA']}, {'start_time': 6.940023, 'end_time': 7.406918, 'objects': ['BENamAV8rkibLIBc8asHhQ', 'XYIgNjEFVU6q-71AupPIzQ']}, {'start_time': 6.917599, 'end_time': 7.651025, 'objects': ['BENamAV8rkibLIBc8asHhQ', '_pI9DAGyIUGSpUuDn8rEHQ']}, {'start_time': 0.511947, 'end_time': 9.961789, 'objects': ['_pI9DAGyIUGSpUuDn8rEHQ', 'XYIgNjEFVU6q-71AupPIzQ']}, {'start_time': 9.895282, 'end_time': 10.273262, 'objects': ['BENamAV8rkibLIBc8asHhQ', '_pI9DAGyIUGSpUuDn8rEHQ']}, {'start_time': 10.472583, 'end_time': 13.094099, 'objects': ['_pI9DAGyIUGSpUuDn8rEHQ', '9jrJzTZvSkqUjwIvPEavYA']}]


In [120]:
for event in touching_situations:
    print(f'Ids: {event["objects"]}, Start time: {event["start_time"]}, End time: {event["end_time"]}')    
    plot_trajectories(event["objects"], event["start_time"], event["end_time"])


Ids: ['AuP-SAqPu02Vrgu6df6Txg', '9jrJzTZvSkqUjwIvPEavYA'], Start time: 0.476154, End time: 0.907067


Ids: ['BENamAV8rkibLIBc8asHhQ', '9jrJzTZvSkqUjwIvPEavYA'], Start time: 0.476154, End time: 1.017507


Ids: ['BENamAV8rkibLIBc8asHhQ', 'XYIgNjEFVU6q-71AupPIzQ'], Start time: 6.940023, End time: 7.406918


Ids: ['BENamAV8rkibLIBc8asHhQ', '_pI9DAGyIUGSpUuDn8rEHQ'], Start time: 6.917599, End time: 7.651025


Ids: ['_pI9DAGyIUGSpUuDn8rEHQ', 'XYIgNjEFVU6q-71AupPIzQ'], Start time: 0.511947, End time: 9.961789


Ids: ['BENamAV8rkibLIBc8asHhQ', '_pI9DAGyIUGSpUuDn8rEHQ'], Start time: 9.895282, End time: 10.273262


Ids: ['_pI9DAGyIUGSpUuDn8rEHQ', '9jrJzTZvSkqUjwIvPEavYA'], Start time: 10.472583, End time: 13.094099
